# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は ID 基準、表示は仕様どおり

## 1) 問題

* `MyNumbers` から **ちょうど1回だけ出現する数（single number）** のうち **最大の数**を 1 行で返す。存在しなければ `NULL` を返す。

* 入力:

  ```
  Table: MyNumbers(num int) -- 重複あり
  ```

* 出力:

  ```
  列: num int  -- single number の最大。存在しなければ NULL
  ```

## 2) 最適解（単一クエリ）

> PostgreSQL では **ウィンドウ COUNT** で single 判定を付与し、最後に `MAX` で 1 行化するのがシンプルかつ高速です（並び不要）。

```sql
WITH win AS (
  SELECT
    num,
    COUNT(*) OVER (PARTITION BY num) AS cnt
  FROM MyNumbers
)
SELECT
  MAX(num) AS num
FROM win
WHERE cnt = 1;

Runtime 190 ms
Beats 67.88%

```

* `cnt = 1` が single。`MAX` の入力が空なら `NULL` になるため要件を満たします。
* `num` に `NULL` が混在しても、`MAX(num)` は `NULL` を無視するため安全。

### 代替（LATERAL で上位 1 件のみを直接取得）

> **インデックス `CREATE INDEX ON MyNumbers(num);`** がある場合、逆順で上位 1 件を取ることで**早期終了**が期待できます。

```sql
-- single な num を降順で 1 件だけ取り出す
SELECT s.num
FROM (VALUES (1)) v(dummy)
CROSS JOIN LATERAL (
  SELECT num
  FROM MyNumbers
  GROUP BY num
  HAVING COUNT(*) = 1
  ORDER BY num DESC
  LIMIT 1
) AS s;

Wrong Answer
13 / 18 testcases passed
```

* 出力は 0 または 1 行。0 行の場合は結果セット空になるので、**常に 1 行（NULL を含む）を返したい**なら次のように包みます：

```sql
SELECT (
  SELECT num
  FROM MyNumbers
  GROUP BY num
  HAVING COUNT(*) = 1
  ORDER BY num DESC
  LIMIT 1
) AS num;

Runtime 196 ms
Beats 54.87%

```

> サブクエリが行を返さなければ `NULL` が返り、要件どおり 1 行固定になります。

## 3) 要点解説

* **方針**:

  1. `num` ごとの出現回数を求める（`COUNT(*) OVER (PARTITION BY num)` または `GROUP BY`）。
  2. `= 1`（single）に絞る。
  3. 最大だけ返す → `MAX(num)` か、`ORDER BY num DESC LIMIT 1` の早期終了。
* **ウィンドウ vs 集約**:

  * 単純集約でも可：`SELECT MAX(num) FROM (SELECT num FROM MyNumbers GROUP BY num HAVING COUNT(*)=1) t;`
  * 大規模でインデックスありなら **`ORDER BY num DESC LIMIT 1`** の形が実務で速いことが多いです。
* **NULL**:

  * `num IS NULL` が single でも、最大値比較では無視される（`MAX` は `NULL` を無視）。
* **インデックス推奨**:

  * `CREATE INDEX ON MyNumbers(num);` により `GROUP BY`/`ORDER BY` が効率化し、`LIMIT 1` で早期終了が期待できます。

## 4) 計算量（概算）

* ウィンドウ版: **O(N log N)** 近辺（パーティション単位のソート/ハッシュ）
* 集約＋`MAX` 版: **O(N)**～**O(N log N)**
* 早期終了（`ORDER BY num DESC LIMIT 1`）: **最良で O(log N)** 探索＋ごく少数検査（分布・インデックス次第）

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 MyNumbers]
  B[出現回数を算出 COUNT OVER or GROUP BY]
  C[cnt = 1 に絞る]
  D[最大だけ取得 MAX or ORDER BY DESC LIMIT 1]
  E[出力 列 num 1行]
  A --> B
  B --> C
  C --> D
  D --> E
```
ナイス計測です！結論から言うと、

* **ウィンドウ版（190ms）は正解＆速い**
* **`ORDER BY ... DESC LIMIT 1` 版の「Wrong Answer」は `NULL` 取り扱いが原因**の可能性が高いです

`PostgreSQL` は `ORDER BY num DESC` だと **`NULL` が先頭に来る（NULLS FIRST）** 挙動です。
そのため「`num IS NULL` がちょうど1回だけある」ケースで、**本来は最大の非NULL single number を返すべきなのに `NULL` を拾ってしまい**誤答になります。

---

## 修正クエリ（早期終了 × 常に1行返す × `NULL` 安全）

```sql
-- 最大の single number を 1 行で返す（なければ NULL）
SELECT (
  SELECT num
  FROM MyNumbers
  GROUP BY num
  HAVING COUNT(*) = 1
  ORDER BY num DESC NULLS LAST  -- ★ ここがポイント
  LIMIT 1
) AS num;

Runtime 195 ms
Beats 56.73%

```

* `NULLS LAST` により、`NULL` は常に末尾へ。

  * 非NULL の single が1つでもあれば、それが先に選ばれます。
  * 非NULLの single が存在しない場合のみ `NULL`（single で唯一なのが `NULL` だけ）を返します。
* サブクエリをスカラで包むので **必ず1行**（`NULL` を含み得る）を返し、ジャッジ仕様に適合。

> もし「`NULL` は single と見なさない」仕様に寄せたいなら、`WHERE num IS NOT NULL` を追加してもOKです：
>
> ```sql
> SELECT (
>   SELECT num
>   FROM MyNumbers
>   WHERE num IS NOT NULL
>   GROUP BY num
>   HAVING COUNT(*) = 1
>   ORDER BY num DESC
>   LIMIT 1
> ) AS num;
> ```

---

## 参考：あなたの2案の所感

* **ウィンドウ版**

```sql
WITH win AS (
SELECT num, COUNT(*) OVER (PARTITION BY num) AS cnt
FROM MyNumbers
)
SELECT MAX(num) AS num
FROM win
WHERE cnt = 1;

Runtime 202 ms
Beats 43.69%

```

  * `MAX` は `NULL` を無視するため、**`NULL` に強い** → 正答性◎。
  * 実測 190ms（67.88%）は良い結果です。

* **LATERAL 版（誤答）**

  * `ORDER BY num DESC` が **`NULL` 先頭**で誤答化。
  * 上記修正（`NULLS LAST` もしくは `WHERE num IS NOT NULL`）で解決。

---

## 追加の微最適化（環境依存）

1. **インデックス**

```sql
CREATE INDEX IF NOT EXISTS ix_mynumbers_num ON MyNumbers(num);
```

* `GROUP BY num` と `ORDER BY num DESC LIMIT 1` のコストを下げ、**早期終了**が効きやすくなります（インデックス逆順走査）。

2. **統計と並列**

```sql
ANALYZE MyNumbers;  -- 統計更新でプラン安定
-- 大きい表なら並列集約が効くことあり（設定：max_parallel_workers_per_gather 等）
```

3. **メモリ**

* `work_mem` を適切に（過度は禁物）→ ハッシュ集約がメモリ内で収まると安定。

---

## まとめ

* 誤答の原因は **`ORDER BY num DESC` の `NULL` 先頭**。
* **`NULLS LAST`**（または `WHERE num IS NOT NULL`）で正答化。
* パフォーマンスは **インデックス + 早期終了**が効く分、修正版のスカラサブクエリが最有力。
* 既存のウィンドウ版（190ms）も十分速く、安定して正しいです。
